In [24]:
import torch
from transformer_lens import HookedTransformer
from transformer_lens import utils

from functools import partial

torch.set_grad_enabled(False)



In [18]:
model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small', device='cpu')


Loaded pretrained model gpt2-small into HookedTransformer


In [11]:
layer = 7
prompt_pos = "Calm"
prompt_neg = "Anger"

In [31]:
logits, cache = model.run_with_cache(prompt_pos)
h_p = cache["resid_post", layer]

logits, cache = model.run_with_cache(prompt_neg)
h_n = cache["resid_post", layer]

steering = h_p - h_n
steering.shape

torch.Size([1, 3, 768])

In [ ]:
def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, position, :] = clean_resid_pre[:, position, :]
    return resid_pre

# We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
num_positions = len(clean_tokens[0])
ioi_patching_result = torch.zeros((model.cfg.n_layers, num_positions), device=model.cfg.device)

for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    for position in range(num_positions):
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = model.run_with_hooks(corrupted_tokens, fwd_hooks=[
            (utils.get_act_name("resid_pre", layer), temp_hook_fn)
        ])
        # Calculate the logit difference
        patched_logit_diff = logits_to_logit_diff(patched_logits).detach()
        # Store the result, normalizing by the clean and corrupted logit difference so it's between 0 and 1 (ish)
        ioi_patching_result[layer, position] = (patched_logit_diff - corrupted_logit_diff)/(clean_logit_diff - corrupted_logit_diff)

In [38]:
c = 5

def residual_stream_patching_hook(
    resid,
    hook,
    layer: int
):
    # resid shape is (batch, pos, d_model)
    len_steering = steering.shape[1]
    print(resid.shape, steering.shape)

    resid[:, :len_steering, :] = resid[:, :len_steering, :] + c * steering
    return resid

In [39]:
text = "I think you're"

hook_name = utils.get_act_name("resid_post", layer)

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, layer=layer))]):
    # model('testing testing asdf;lkjasdf;l k')
    model.generate(text, prepend_bos=True)


  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([1, 5, 768]) torch.Size([1, 3, 768])
torch.Size([1, 1, 768]) torch.Size([1, 3, 768])


RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 1.  Target sizes: [1, 1, 768].  Tensor sizes: [3, 768]